## Pretrained VGGNet

In [1]:
# Operations for floydhub
!git clone https://github.com/machrisaa/tensorflow-vgg tensorflow_vgg
!ln -s /data .

Cloning into 'tensorflow_vgg'...
remote: Counting objects: 113, done.
remote: Total 113 (delta 0), reused 0 (delta 0), pack-reused 113
Receiving objects: 100% (113/113), 55.91 KiB | 0 bytes/s, done.
Resolving deltas: 100% (61/61), done.
Checking connectivity... done.


In [1]:
from urllib.request import urlretrieve
from os.path import isfile, isdir
from tqdm import tqdm

vgg_dir = 'tensorflow_vgg/'
vgg_name = 'vgg19'

# Make sure vgg exists
if not isdir(vgg_dir):
    raise Exception("VGG directory doesn't exist!")

class DLProgress(tqdm):
    last_block = 0

    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num

vgg_param_file = '{}{}.npy'.format(vgg_dir, vgg_name)
if not isfile(vgg_param_file):
    with DLProgress(unit='B', unit_scale=True, miniters=1, desc= vgg_name + ' Parameters') as pbar:
        urlretrieve(
            'https://s3.amazonaws.com/content.udacity-data.com/nd101/{}.npy'.format(vgg_name),
            vgg_param_file,
            pbar.hook)
else:
    print("Parameter file already exists!")

Parameter file already exists!


In [2]:
import os

import numpy as np
import tensorflow as tf

# from tensorflow_vgg import vgg19
from tensorflow_vgg import vgg16
from tensorflow_vgg import utils

In [3]:
data_dir = 'data/'
train_dir = data_dir + 'train/'

classes = [d for d in os.listdir(train_dir) if os.path.isdir(train_dir + d)]

In [ ]:
from scipy.ndimage.interpolation import rotate
from skimage.transform import resize

def horizontal_flip(image):
    image = image[:, ::-1, :]
    return image

def vertical_flip(image):
    image = image[::-1, :, :]
    return image

def random_rotation(image, angle_range=(0, 180)):
    h, w, _ = image.shape
    angle = np.random.randint(*angle_range)
    image = rotate(image, angle)
    image = resize(image, (h, w))
    return image

In [ ]:
import csv

batch_size = 10
batch = []

with tf.Session() as sess:
    # vgg = vgg19.Vgg19()
    vgg = vgg16.Vgg16()
    input_ = tf.placeholder(tf.float32, [None, 224, 224, 3])
    
    with tf.name_scope("content_vgg"):
        vgg.build(input_)
    
    
    codes = None
    labels = []
    count = 0

    for d_type in ['train', 'valid', 'test']:
        for c in classes:
            image_dir = '{}{}/{}/'.format(data_dir, d_type, c) # e.g. data/train/melanoma/
            files = os.listdir(image_dir)
            for i, file in enumerate(files, 1):
                # load image and resize it to 224x224
                file_path = os.path.join(image_dir, file)
                img = utils.load_image(file_path)
                batch.append(img.reshape((1, 224, 224, 3)))
                labels.append(c)
                
                # data augumentation for training data
                if d_type == 'train':
                    batch.append(horizontal_flip(img).reshape((1,224,224,3)))
                    labels.append(c)
                    batch.append(vertical_flip(img).reshape((1,224,224,3)))
                    labels.append(c)
                    batch.append(random_rotation(img).reshape((1,224,224,3)))
                    labels.append(c)
                    batch.append(random_rotation(img, angle_range=(-180, 0)).reshape((1,224,224,3)))
                    labels.append(c)                  

                if (len(batch) >= batch_size) or i == len(files):
                    images = np.concatenate(batch)

                    feed_dict = {input_: images}
                    codes_batch = sess.run(vgg.relu6, feed_dict=feed_dict)

                    if codes is None:
                        codes = codes_batch
                    else:
                        codes = np.concatenate((codes, codes_batch))

                    count += len(batch)
                    batch = []
                    print('data: {}, class: {}, {} / {} images processed. data count: {}'.format(d_type, c, i, len(files), count))

        # write codes to file
        with open('{}_{}_codes'.format(vgg_name, d_type), 'w') as f:
            codes.tofile(f)
            codes = None

        # write labels to file
        with open('{}_{}_labels'.format(vgg_name, d_type), 'w') as f:
            writer = csv.writer(f, delimiter='\n')
            writer.writerow(labels)
            labels = []

/Users/junji/Development/udacity-deeplearning/dermatologist-ai/tensorflow_vgg/vgg16.npy
npy file loaded
build model started
build model finished: 0s


/Users/junji/miniconda3/envs/dermatologist-ai/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/Users/junji/miniconda3/envs/dermatologist-ai/lib/python3.5/site-packages/ipykernel_launcher.py:16: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  app.launch_new_instance()


## Building the Classifier

In [30]:
# read codes and labels from file
import csv

# train data
with open('{}_train_labels'.format(vgg_name)) as f:
    reader = csv.reader(f, delimiter='\n')
    train_labels = np.array([each for each in reader if len(each) > 0]).squeeze()
with open('{}_train_codes'.format(vgg_name)) as f:
    train_x = np.fromfile(f, dtype=np.float32)
    train_x = train_x.reshape((len(train_labels), -1))
    
# valid data
with open('{}_valid_labels'.format(vgg_name)) as f:
    reader = csv.reader(f, delimiter='\n')
    valid_labels = np.array([each for each in reader if len(each) > 0]).squeeze()
with open('{}_valid_codes'.format(vgg_name)) as f:
    val_x = np.fromfile(f, dtype=np.float32)
    val_x = val_x.reshape((len(valid_labels), -1))
    
# test data
with open('{}_test_labels'.format(vgg_name)) as f:
    reader = csv.reader(f, delimiter='\n')
    test_labels = np.array([each for each in reader if len(each) > 0]).squeeze()
with open('{}_test_codes'.format(vgg_name)) as f:
    test_x = np.fromfile(f, dtype=np.float32)
    test_x = test_x.reshape((len(test_labels), -1))

In [47]:
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()
lb.fit(classes)

train_y = lb.transform(train_labels)
val_y = lb.transform(valid_labels)
test_y = lb.transform(test_labels)

In [48]:
print("Train shapes (x, y):", train_x.shape, train_y.shape)
print("Validation shapes (x, y):", val_x.shape, val_y.shape)
print("Test shapes (x, y):", test_x.shape, test_y.shape)

Train shapes (x, y): (2000, 4096) (2000, 3)
Validation shapes (x, y): (150, 4096) (150, 3)
Test shapes (x, y): (600, 4096) (600, 3)


In [60]:
inputs_ = tf.placeholder(tf.float32, shape=[None, train_x.shape[1]])
labels_ = tf.placeholder(tf.int64, shape=[None, train_y.shape[1]])

fc_1 = tf.contrib.layers.fully_connected(inputs_, 512)
dropout_1 = tf.contrib.layers.dropout(fc_1)
fc_2 = tf.contrib.layers.fully_connected(dropout_1, 128)
dropout_2 = tf.contrib.layers.dropout(fc_2)
    
logits = tf.contrib.layers.fully_connected(dropout_2, train_y.shape[1], activation_fn=None)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=labels_, logits=logits)
cost = tf.reduce_mean(cross_entropy)

optimizer = tf.train.AdamOptimizer(learning_rate=0.0005).minimize(cost)

predicted = tf.nn.softmax(logits)
correct_pred = tf.equal(tf.argmax(predicted, 1), tf.argmax(labels_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [31]:
def get_batches(x, y, n_batches=10):
    """ Return a generator that yields batches from arrays x and y. """
    batch_size = len(x)//n_batches
    
    for ii in range(0, n_batches*batch_size, batch_size):
        # If we're not on the last batch, grab data with size batch_size
        if ii != (n_batches-1)*batch_size:
            X, Y = x[ii: ii+batch_size], y[ii: ii+batch_size] 
        # On the last batch, grab the rest of the data
        else:
            X, Y = x[ii:], y[ii:]
        # I love generators
        yield X, Y

## Training

In [79]:
!mkdir checkpoints

epochs = 500
iteration = 0
saver = tf.train.Saver()
with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    # saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    for e in range(epochs):
        for x, y in get_batches(train_x, train_y, n_batches=5):
            feed = {inputs_: x,
                    labels_: y}
            loss, _ = sess.run([cost, optimizer], feed_dict=feed)
            print("Epoch: {}/{}".format(e+1, epochs),
                  "Iteration: {}".format(iteration),
                  "Training loss: {:.5f}".format(loss))
            iteration += 1
            
            if iteration % 5 == 0:
                feed = {inputs_: val_x, labels_: val_y}
                val_acc = sess.run(accuracy, feed_dict=feed)
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Validation Acc: {:.4f}".format(val_acc))
    saver.save(sess, "checkpoints/skin_diseases.ckpt")

mkdir: cannot create directory ‘checkpoints’: File exists
Epoch: 1/500 Iteration: 0 Training loss: 8.01568
Epoch: 1/500 Iteration: 1 Training loss: 6.44547
Epoch: 1/500 Iteration: 2 Training loss: 0.55847
Epoch: 1/500 Iteration: 3 Training loss: 0.04495
Epoch: 1/500 Iteration: 4 Training loss: 50.26734
Epoch: 0/500 Iteration: 5 Validation Acc: 0.5200
Epoch: 2/500 Iteration: 5 Training loss: 17.40879
Epoch: 2/500 Iteration: 6 Training loss: 0.04064
Epoch: 2/500 Iteration: 7 Training loss: 0.05108
Epoch: 2/500 Iteration: 8 Training loss: 0.15226
Epoch: 2/500 Iteration: 9 Training loss: 14.76763
Epoch: 1/500 Iteration: 10 Validation Acc: 0.5067
Epoch: 3/500 Iteration: 10 Training loss: 4.36388
Epoch: 3/500 Iteration: 11 Training loss: 1.37924
Epoch: 3/500 Iteration: 12 Training loss: 1.79588
Epoch: 3/500 Iteration: 13 Training loss: 2.30511
Epoch: 3/500 Iteration: 14 Training loss: 4.15821
Epoch: 2/500 Iteration: 15 Validation Acc: 0.3733
Epoch: 4/500 Iteration: 15 Training loss: 3.14561


Epoch: 29/500 Iteration: 150 Validation Acc: 0.5867
Epoch: 31/500 Iteration: 150 Training loss: 0.80016
Epoch: 31/500 Iteration: 151 Training loss: 0.34255
Epoch: 31/500 Iteration: 152 Training loss: 0.34326
Epoch: 31/500 Iteration: 153 Training loss: 0.38091
Epoch: 31/500 Iteration: 154 Training loss: 1.29398
Epoch: 30/500 Iteration: 155 Validation Acc: 0.5933
Epoch: 32/500 Iteration: 155 Training loss: 0.82281
Epoch: 32/500 Iteration: 156 Training loss: 0.39666
Epoch: 32/500 Iteration: 157 Training loss: 0.37757
Epoch: 32/500 Iteration: 158 Training loss: 0.37707
Epoch: 32/500 Iteration: 159 Training loss: 1.20556
Epoch: 31/500 Iteration: 160 Validation Acc: 0.6067
Epoch: 33/500 Iteration: 160 Training loss: 0.83828
Epoch: 33/500 Iteration: 161 Training loss: 0.36699
Epoch: 33/500 Iteration: 162 Training loss: 0.34710
Epoch: 33/500 Iteration: 163 Training loss: 0.35048
Epoch: 33/500 Iteration: 164 Training loss: 1.14363
Epoch: 32/500 Iteration: 165 Validation Acc: 0.6467
Epoch: 34/50

Epoch: 58/500 Iteration: 289 Training loss: 0.90487
Epoch: 57/500 Iteration: 290 Validation Acc: 0.6667
Epoch: 59/500 Iteration: 290 Training loss: 0.48871
Epoch: 59/500 Iteration: 291 Training loss: 0.24844
Epoch: 59/500 Iteration: 292 Training loss: 0.25698
Epoch: 59/500 Iteration: 293 Training loss: 0.21937
Epoch: 59/500 Iteration: 294 Training loss: 0.86012
Epoch: 58/500 Iteration: 295 Validation Acc: 0.6600
Epoch: 60/500 Iteration: 295 Training loss: 0.50163
Epoch: 60/500 Iteration: 296 Training loss: 0.26016
Epoch: 60/500 Iteration: 297 Training loss: 0.24302
Epoch: 60/500 Iteration: 298 Training loss: 0.24443
Epoch: 60/500 Iteration: 299 Training loss: 0.78034
Epoch: 59/500 Iteration: 300 Validation Acc: 0.6467
Epoch: 61/500 Iteration: 300 Training loss: 0.46607
Epoch: 61/500 Iteration: 301 Training loss: 0.21671
Epoch: 61/500 Iteration: 302 Training loss: 0.21545
Epoch: 61/500 Iteration: 303 Training loss: 0.22016
Epoch: 61/500 Iteration: 304 Training loss: 0.83865
Epoch: 60/50

Epoch: 87/500 Iteration: 434 Training loss: 0.53709
Epoch: 86/500 Iteration: 435 Validation Acc: 0.6267
Epoch: 88/500 Iteration: 435 Training loss: 0.28612
Epoch: 88/500 Iteration: 436 Training loss: 0.15251
Epoch: 88/500 Iteration: 437 Training loss: 0.13637
Epoch: 88/500 Iteration: 438 Training loss: 0.12868
Epoch: 88/500 Iteration: 439 Training loss: 0.48542
Epoch: 87/500 Iteration: 440 Validation Acc: 0.6933
Epoch: 89/500 Iteration: 440 Training loss: 0.24604
Epoch: 89/500 Iteration: 441 Training loss: 0.12488
Epoch: 89/500 Iteration: 442 Training loss: 0.14139
Epoch: 89/500 Iteration: 443 Training loss: 0.09948
Epoch: 89/500 Iteration: 444 Training loss: 0.51406
Epoch: 88/500 Iteration: 445 Validation Acc: 0.6667
Epoch: 90/500 Iteration: 445 Training loss: 0.25758
Epoch: 90/500 Iteration: 446 Training loss: 0.12573
Epoch: 90/500 Iteration: 447 Training loss: 0.14764
Epoch: 90/500 Iteration: 448 Training loss: 0.12845
Epoch: 90/500 Iteration: 449 Training loss: 0.49613
Epoch: 89/50

Epoch: 116/500 Iteration: 576 Training loss: 0.06978
Epoch: 116/500 Iteration: 577 Training loss: 0.07101
Epoch: 116/500 Iteration: 578 Training loss: 0.07932
Epoch: 116/500 Iteration: 579 Training loss: 0.29200
Epoch: 115/500 Iteration: 580 Validation Acc: 0.6800
Epoch: 117/500 Iteration: 580 Training loss: 0.13300
Epoch: 117/500 Iteration: 581 Training loss: 0.07158
Epoch: 117/500 Iteration: 582 Training loss: 0.08458
Epoch: 117/500 Iteration: 583 Training loss: 0.07524
Epoch: 117/500 Iteration: 584 Training loss: 0.29108
Epoch: 116/500 Iteration: 585 Validation Acc: 0.6933
Epoch: 118/500 Iteration: 585 Training loss: 0.12594
Epoch: 118/500 Iteration: 586 Training loss: 0.06089
Epoch: 118/500 Iteration: 587 Training loss: 0.06786
Epoch: 118/500 Iteration: 588 Training loss: 0.06580
Epoch: 118/500 Iteration: 589 Training loss: 0.28232
Epoch: 117/500 Iteration: 590 Validation Acc: 0.7067
Epoch: 119/500 Iteration: 590 Training loss: 0.16448
Epoch: 119/500 Iteration: 591 Training loss: 0

...

Epoch: 367/500 Iteration: 1833 Training loss: 0.26182
Epoch: 367/500 Iteration: 1834 Training loss: 0.81790
Epoch: 366/500 Iteration: 1835 Validation Acc: 0.6267
Epoch: 368/500 Iteration: 1835 Training loss: 0.41805
Epoch: 368/500 Iteration: 1836 Training loss: 0.23339
Epoch: 368/500 Iteration: 1837 Training loss: 0.28184
Epoch: 368/500 Iteration: 1838 Training loss: 0.22595
Epoch: 368/500 Iteration: 1839 Training loss: 0.45699
Epoch: 367/500 Iteration: 1840 Validation Acc: 0.6467
Epoch: 369/500 Iteration: 1840 Training loss: 0.34996
Epoch: 369/500 Iteration: 1841 Training loss: 0.18486
Epoch: 369/500 Iteration: 1842 Training loss: 0.20129
Epoch: 369/500 Iteration: 1843 Training loss: 0.16447
Epoch: 369/500 Iteration: 1844 Training loss: 0.58118
Epoch: 368/500 Iteration: 1845 Validation Acc: 0.6733
Epoch: 370/500 Iteration: 1845 Training loss: 0.36183
Epoch: 370/500 Iteration: 1846 Training loss: 0.18518
Epoch: 370/500 Iteration: 1847 Training loss: 0.23303
Epoch: 370/500 Iteration: 18

Epoch: 393/500 Iteration: 1960 Training loss: 0.18574
Epoch: 393/500 Iteration: 1961 Training loss: 0.07040
Epoch: 393/500 Iteration: 1962 Training loss: 0.07400
Epoch: 393/500 Iteration: 1963 Training loss: 0.08649
Epoch: 393/500 Iteration: 1964 Training loss: 0.17507
Epoch: 392/500 Iteration: 1965 Validation Acc: 0.6933
Epoch: 394/500 Iteration: 1965 Training loss: 0.14539
Epoch: 394/500 Iteration: 1966 Training loss: 0.09650
Epoch: 394/500 Iteration: 1967 Training loss: 0.05690
Epoch: 394/500 Iteration: 1968 Training loss: 0.04755
Epoch: 394/500 Iteration: 1969 Training loss: 0.32520
Epoch: 393/500 Iteration: 1970 Validation Acc: 0.6867
Epoch: 395/500 Iteration: 1970 Training loss: 0.15613
Epoch: 395/500 Iteration: 1971 Training loss: 0.07897
Epoch: 395/500 Iteration: 1972 Training loss: 0.09435
Epoch: 395/500 Iteration: 1973 Training loss: 0.12596
Epoch: 395/500 Iteration: 1974 Training loss: 0.19151
Epoch: 394/500 Iteration: 1975 Validation Acc: 0.7067
Epoch: 396/500 Iteration: 19

Epoch: 421/500 Iteration: 2104 Training loss: 0.12660
Epoch: 420/500 Iteration: 2105 Validation Acc: 0.6733
Epoch: 422/500 Iteration: 2105 Training loss: 0.08282
Epoch: 422/500 Iteration: 2106 Training loss: 0.03473
Epoch: 422/500 Iteration: 2107 Training loss: 0.04913
Epoch: 422/500 Iteration: 2108 Training loss: 0.04663
Epoch: 422/500 Iteration: 2109 Training loss: 0.21524
Epoch: 421/500 Iteration: 2110 Validation Acc: 0.6933
Epoch: 423/500 Iteration: 2110 Training loss: 0.08825
Epoch: 423/500 Iteration: 2111 Training loss: 0.04698
Epoch: 423/500 Iteration: 2112 Training loss: 0.04015
Epoch: 423/500 Iteration: 2113 Training loss: 0.04033
Epoch: 423/500 Iteration: 2114 Training loss: 0.09130
Epoch: 422/500 Iteration: 2115 Validation Acc: 0.6467
Epoch: 424/500 Iteration: 2115 Training loss: 0.07489
Epoch: 424/500 Iteration: 2116 Training loss: 0.04642
Epoch: 424/500 Iteration: 2117 Training loss: 0.03745
Epoch: 424/500 Iteration: 2118 Training loss: 0.03700
Epoch: 424/500 Iteration: 21

Epoch: 447/500 Iteration: 2231 Training loss: 0.02470
Epoch: 447/500 Iteration: 2232 Training loss: 0.03746
Epoch: 447/500 Iteration: 2233 Training loss: 0.03325
Epoch: 447/500 Iteration: 2234 Training loss: 0.10507
Epoch: 446/500 Iteration: 2235 Validation Acc: 0.6733
Epoch: 448/500 Iteration: 2235 Training loss: 0.10241
Epoch: 448/500 Iteration: 2236 Training loss: 0.03678
Epoch: 448/500 Iteration: 2237 Training loss: 0.02409
Epoch: 448/500 Iteration: 2238 Training loss: 0.01360
Epoch: 448/500 Iteration: 2239 Training loss: 0.11253
Epoch: 447/500 Iteration: 2240 Validation Acc: 0.6467
Epoch: 449/500 Iteration: 2240 Training loss: 0.06620
Epoch: 449/500 Iteration: 2241 Training loss: 0.03325
Epoch: 449/500 Iteration: 2242 Training loss: 0.02529
Epoch: 449/500 Iteration: 2243 Training loss: 0.03857
Epoch: 449/500 Iteration: 2244 Training loss: 0.10308
Epoch: 448/500 Iteration: 2245 Validation Acc: 0.6733
Epoch: 450/500 Iteration: 2245 Training loss: 0.07756
Epoch: 450/500 Iteration: 22

Epoch: 476/500 Iteration: 2375 Training loss: 0.05240
Epoch: 476/500 Iteration: 2376 Training loss: 0.04109
Epoch: 476/500 Iteration: 2377 Training loss: 0.02145
Epoch: 476/500 Iteration: 2378 Training loss: 0.01740
Epoch: 476/500 Iteration: 2379 Training loss: 0.14797
Epoch: 475/500 Iteration: 2380 Validation Acc: 0.6667
Epoch: 477/500 Iteration: 2380 Training loss: 0.07942
Epoch: 477/500 Iteration: 2381 Training loss: 0.02105
Epoch: 477/500 Iteration: 2382 Training loss: 0.02211
Epoch: 477/500 Iteration: 2383 Training loss: 0.02477
Epoch: 477/500 Iteration: 2384 Training loss: 0.05111
Epoch: 476/500 Iteration: 2385 Validation Acc: 0.7067
Epoch: 478/500 Iteration: 2385 Training loss: 0.05960
Epoch: 478/500 Iteration: 2386 Training loss: 0.02956
Epoch: 478/500 Iteration: 2387 Training loss: 0.02350
Epoch: 478/500 Iteration: 2388 Training loss: 0.02398
Epoch: 478/500 Iteration: 2389 Training loss: 0.13902
Epoch: 477/500 Iteration: 2390 Validation Acc: 0.7000
Epoch: 479/500 Iteration: 23

## Testing

In [78]:
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    feed = {inputs_: test_x,
            labels_: test_y}
    test_acc = sess.run(accuracy, feed_dict=feed)
    print("Test accuracy: {:.4f}".format(test_acc))

INFO:tensorflow:Restoring parameters from checkpoints/skin_diseases.ckpt
Test accuracy: 0.6750
